# 📘 Notebook RAG com CSV: Análise de Reviews com Busca Semântica
Este notebook guia a criação de embeddings para comentários de clientes e busca por similaridade usando FAISS.

**Dataset**: `olist_order_reviews_dataset.csv`

🔁 Leitura → Limpeza → Embeddings → Indexação → Busca

## 📥 Etapa 1: Leitura e limpeza dos dados

In [16]:
import pandas as pd

# Leitura do arquivo CSV
df = pd.read_csv("data/olist_order_reviews_dataset.csv", encoding="utf-8")

# Limpar e manter apenas mensagens de review
df_clean = df[['review_comment_message']].dropna()
df_clean = df_clean[df_clean['review_comment_message'].str.strip() != '']
df_clean.reset_index(drop=True, inplace=True)

print(f'Total de comentários limpos: {len(df_clean)}')
df_clean.head()

Total de comentários limpos: 40950


,review_comment_message
0,Recebi bem antes do prazo estipulado.
1,Parabéns lojas lannister adorei comprar pela I...
2,aparelho eficiente. no site a marca do aparelh...
3,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,"Vendedor confiável, produto ok e entrega antes..."


## 🧠 Etapa 2: Criar embeddings dos reviews

In [11]:
from sentence_transformers import SentenceTransformer

# Carregar modelo leve e eficiente
modelo = SentenceTransformer('all-MiniLM-L6-v2')

# Gerar vetores para os comentários
comentarios = df_clean['review_comment_message'].tolist()
vetores = modelo.encode(comentarios, show_progress_bar=True)

print("Embeddings criados!", vetores.shape)

Batches: 100%|██████████| 1280/1280 [03:30<00:00,  6.07it/s]


Embeddings criados! (40950, 384)


## 🔎 Etapa 3: Indexar os embeddings com FAISS

In [17]:
import faiss
import numpy as np

# Criar índice FAISS com vetores
dim = vetores.shape[1]
indice = faiss.IndexFlatL2(dim)
indice.add(np.array(vetores))

# Salvar índice para reuso futuro (opcional)
faiss.write_index(indice, "indice_reviews.faiss")

## 🔁 Etapa 4: Buscar comentários semelhantes

In [18]:
def buscar_reviews_similares(texto: str, top_k: int = 3):
    vetor_consulta = modelo.encode([texto])
    _, indices = indice.search(np.array(vetor_consulta), top_k)
    return df_clean.iloc[indices[0]]

# Exemplo de teste:
buscar_reviews_similares("Produto com defeito e entrega atrasada")

,review_comment_message
21981,O produto veio com defeito e atraso na entrega.
18725,Produto com entrega atrasada
31047,Produto com o prazo de entrega atrasado
